In [1]:
import numpy as np
import pandas as pd
import PIL
import os
pd.options.mode.chained_assignment = None  # default='warn'
from dataloader import CNNLSTMDataLoader

In [2]:
csv_path = 'progressiondataset.csv'
img_path = '/home/vip/sayan-mandal/datasets/obj_criteria/good_reduced/'
dl = CNNLSTMDataLoader(csv_path, img,path)

In [3]:
dl[0]

(['al0320_2005_02_16_os.jpg',
  'al0320_2006_01_13_os.jpg',
  'al0320_2007_03_26_os.jpg'],
 -0.20578472,
 3)

In [4]:
dataset = pd.read_csv('progressiondataset.csv')

In [6]:
timestep = 3
grouped = dataset.groupby('maskedeye')

In [7]:
grouped = iter(grouped)
_, group = next(grouped)


In [30]:
agg_func = {}
for col in ['filename','maskedeye','slope']:
    agg_func[col] = 'last'

In [31]:
new = []
for _, group in grouped:
    group = group.sort_values(by=['pdate'],ascending=True)
    group = group.groupby(group['pdate']).aggregate(agg_func).reset_index().reindex(columns=group.columns)
    new += [group]

In [35]:
new_df = pd.concat(new).reset_index().reindex(columns=new[0].columns)

In [36]:
new_df.to_csv('progressiondataset.csv', index = False)

In [12]:
group = group.sort_values(by=['pdate'],ascending=True)
group = group.groupby(group['pdate']).aggregate(agg_func).reset_index().reindex(columns=group.columns)


In [13]:
group

,filename,maskedeye,pdate,slope
0,al0320_2005_02_16_os.jpg,al0320l,2005-02-16,-0.205785
1,al0320_2006_01_13_os.jpg,al0320l,2006-01-13,-0.205785
2,al0320_2007_03_26_os.jpg,al0320l,2007-03-26,-0.205785
3,al0320_2011_04_26_os_01.jpg,al0320l,2011-04-26,-0.205785
4,al0320_2012_07_25_os_01.jpg,al0320l,2012-07-25,-0.205785
5,al0320_2014_02_18_os_01.jpg,al0320l,2014-02-18,-0.205785
6,al0320_2015_03_19_os_01.jpg,al0320l,2015-03-19,-0.205785


In [8]:
if len(group) == 3:
    print(group['filename'].iloc[:3].to_list(), group['slope'].iloc[3-1])
elif len(group) > 3:
    for i in range(3, group.shape[0]+1):
        if i < timestep:
            print(group['filename'].iloc[:i].to_list(), group['slope'].iloc[i-1])
        else:
            print(group['filename'].iloc[i-timestep:i].to_list(), group['slope'].iloc[i-1])

['al0320_2005_02_16_os.jpg', 'al0320_2006_01_13_os.jpg', 'al0320_2007_03_26_os.jpg'] -0.20578472
['al0320_2006_01_13_os.jpg', 'al0320_2007_03_26_os.jpg', 'al0320_2011_04_26_os_01.jpg'] -0.20578472
['al0320_2007_03_26_os.jpg', 'al0320_2011_04_26_os_01.jpg', 'al0320_2012_07_25_os_01.jpg'] -0.20578472
['al0320_2011_04_26_os_01.jpg', 'al0320_2012_07_25_os_01.jpg', 'al0320_2014_02_18_os_01.jpg'] -0.20578472
['al0320_2012_07_25_os_01.jpg', 'al0320_2014_02_18_os_01.jpg', 'al0320_2015_03_19_os_01.jpg'] -0.20578472


In [23]:
group.iloc[2]

filename     al0320_2007_03_26_os.jpg
maskedeye                     al0320l
pdate                      2007-03-26
slope                       -0.205785
Name: 2, dtype: object

In [4]:
chunks = []
labels = []
seqlen = []
for _, group in grouped:
    group = group.sort_values(by=['day'],ascending=False)
    if len(group) == self.n:
            block = torch.tensor(group.iloc[-self.n:,1:-1].to_numpy())
            chunks += [torch.flip(torch.nn.functional.pad(block,(0,0,0,self.timestep-block.shape[0])),[0])]
            labels += [self.encode(group[labelcol].iloc[0])]
            seqlen += [self.n]
    elif len(group)>self.n:
        for i in range(self.n,group.shape[0]+1):
            block = torch.tensor(group.iloc[-i:,1:-1].to_numpy())
            chunks += [torch.flip(torch.nn.functional.pad(block,(0,0,0,self.timestep-block.shape[0])),[0])]
            labels += [self.encode(group[labelcol].iloc[-i])]
            seqlen += [i]

In [5]:
df = df[(df['scantype'] != "OCT Radial Circle Scan") & (df['diametermm'] < 4) & (df['age'] > 18) & 
            (df['age'] < 100) & (df['quality'] >15) & (df['rnflmean_g'] > 20) & (df['rnflmean_g'] < 150) &
            (~df['crt1'].isin(criteria1)) & (~df['crt2'].isin(criteria2)) & (~df['crt3'].isin(criteria3))]


df = df.drop(columns=['age','crt1','crt2','crt3'])

df = df.loc[(df[cols[29:-1]] != 0).all(axis=1)]
df = df.drop(columns=['dob_shift', 'mergedby', 'imageid', 'scantype', 'examtime', 'aqmversion', 'rnflclass_g', 'rnflclass_t', 'rnflclass_ts', 'rnflclass_ti', 'rnflclass_n', 'rnflclass_ns', 'rnflclass_ni','satellite'])

In [6]:
cols1 = df.columns.to_list()
agg_func = {}
for col in cols1[:2]+cols1[3:4]:
    agg_func[col] = 'first'
for col in cols1[4:]:
    agg_func[col] = 'mean'

In [7]:
grouped = df.groupby('eye')
grouped = iter(grouped)
_, df_Leye = next(grouped)
_, df_Reye = next(grouped)
df_Reye = df_Reye.reset_index().drop(columns=['index'])
df_Leye = df_Leye.reset_index().drop(columns=['index'])

In [8]:
grouped = df_Reye.groupby('maskedid')

singlevals = []

appenddata = []
for name, tempdf in grouped:
    if tempdf.shape[0] == 1:
        tempdf['days'] = (tempdf['examdate_shift'] - tempdf['examdate_shift'].iloc[0]).dt.days
        tempdf['progress'] = 999.0
        tempdf['label'] = 'b'
        singlevals+= [name]
    else:
        tempdf = tempdf.groupby(tempdf[cols1[2]]).aggregate(agg_func).reset_index().reindex(columns=tempdf.columns)
        if tempdf.shape[0] == 1:
            tempdf['days'] = (tempdf['examdate_shift'] - tempdf['examdate_shift'].iloc[0]).dt.days
            tempdf['progress'] = 999.0
            tempdf['label'] = 'b'
            singlevals+= [name]
        else:
            tempdf['days'] = (tempdf['examdate_shift'] - tempdf['examdate_shift'].iloc[0]).dt.days
            tempdf['progress'] = 999.0
            tempdf['progress'].iloc[1:] = tempdf['rnflmean_g'].iloc[1:] - tempdf['rnflmean_g'].shift().iloc[1:]
            tempdf['label'] = tempdf["progress"].apply(lambda x: 'b' if x == 999.0  else ('p' if x < 0 else 'n'))
    
    appenddata += [tempdf]
n_dfR = pd.concat(appenddata)
    

In [9]:
grouped = df_Leye.groupby('maskedid')

appenddata = []
for name, tempdf in grouped:
    if tempdf.shape[0] == 1:
        tempdf['days'] = (tempdf['examdate_shift'] - tempdf['examdate_shift'].iloc[0]).dt.days
        tempdf['progress'] = 999.0
        tempdf['label'] = 'b'
        singlevals+= [name]
    else:
        tempdf = tempdf.groupby(tempdf[cols1[2]]).aggregate(agg_func).reset_index().reindex(columns=tempdf.columns)
        if tempdf.shape[0] == 1:
            tempdf['days'] = (tempdf['examdate_shift'] - tempdf['examdate_shift'].iloc[0]).dt.days
            tempdf['progress'] = 999.0
            tempdf['label'] = 'b'
            singlevals+= [name]
        else:
            tempdf['days'] = (tempdf['examdate_shift'] - tempdf['examdate_shift'].iloc[0]).dt.days
            tempdf['progress'] = 999.0
            tempdf['progress'].iloc[1:] = tempdf['rnflmean_g'].iloc[1:] - tempdf['rnflmean_g'].shift().iloc[1:]
            tempdf['label'] = tempdf["progress"].apply(lambda x: 'b' if x == 999.0  else ('p' if x < 0 else 'n'))
    
    appenddata += [tempdf]
n_dfL = pd.concat(appenddata)

In [10]:
df_comb = pd.concat([n_dfR,n_dfL])
df_comb['age'] = (df_comb['examdate_shift'] - df_comb['dob_deduce_shift']).dt.days/365.25
df_comb = df_comb.reindex(columns=cols1[:4]+['days','age','progress']+cols1[4:]+['label'])


In [11]:
df_comb.to_csv('octdataset.csv')

In [16]:
df_comb['progress'][df_comb['progress']==999.0] = 0.0

In [18]:
new_df = df_comb.reset_index().drop(columns=['index'])
new_df.to_csv('octdataset.csv')